In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import mediapipe as mp
from random import sample

In [ ]:
# Salva 21 landmarks da mão para cada coordenada de x, y e z
def extract_landmarks(results):
    left_hand = np.array([[result_left.x, result_left.y, result_left.z] for result_left in results.left_hand_landmarks.landmark]) if results.left_hand_landmarks else np.zeros((21,3))
    right_hand = np.array([[result_right.x, result_right.y, result_right.z] for result_right in results.right_hand_landmarks.landmark]) if results.right_hand_landmarks else np.zeros((21,3))
    hands = np.concatenate([left_hand.flatten(), right_hand.flatten()])
    return hands


In [ ]:
# Leitura com vídeos de entrada 
# --- Classe 1
# --------Vídeo 1
# --------Video 2

img_size = 224, 224
max_frames = 20
data_dir = 'dataset' # diretório de entrada 


# Cria um diretório de saída
data_points = "dataset_videos_landmarks"
if not os.path.exists(data_points):
    os.makedirs(data_points)


mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    for sign_folder in sorted(os.listdir(data_dir)): 
        print(sign_folder)
        
        if not os.path.exists(os.path.join(data_points, sign_folder)):
            os.mkdir(os.path.join(data_points, sign_folder))
        
        for video_folder in sorted(os.listdir(data_dir + '/' + sign_folder)): 

            cap = cv2.VideoCapture(data_dir + '/' +  sign_folder + '/' +  video_folder)
            frames = []
            try:
                while True:
                    ret, frame = cap.read()
                    if not ret:
                        break

                    frame = cv2.resize(frame, img_size)
                    frames.append(frame)

            finally:
                cap.release()


            if len(frames) > max_frames and max_frames!=0: 
                qtd_frames = sorted(sample(range(0, len(frames)), max_frames))

                for i in sorted(qtd_frames):
                    frames[i].flags.writeable = False                  # Image is no longer writeable
                    results = holistic.process(frames[i])              # Make prediction
                    keypoints = extract_landmarks(results)         # extract landmarks 
        
        
                    if not os.path.exists(os.path.join(data_points, sign_folder, video_folder.split(".mp4")[0])):
                        os.mkdir(os.path.join(data_points, sign_folder, video_folder.split(".mp4")[0]))
                    
                    npy_path = os.path.join(data_points, sign_folder, video_folder.split(".mp4")[0], str(i))
                    np.save(npy_path, keypoints)




